<img style="float: left;" width="20%" src="pics/escudo_ubu.png">
<br style="clear: both;" />

# Minería de datos

<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 5px;">Práctica 2: ENN y KNN</h2>

## Docentes

- Mario Juez Gil (mariojg@ubu.es)

<br />
**Materiales realizados por:** Mario Juez Gil y José Francisco Diez Pastor

<a id="index"></a>
## Tabla de contenidos del notebook

1. [Introducción al aprendizaje supervisado](#intro)
2. [Vecinos más cercanos o k-NN](#vecinos)
3. [Selección de instancias](#instance-selection)
4. [Tareas de la practica k-NN](#tareas)
3. [Condiciones de entrega](#entrega)

# Introducción al aprendizaje supervisado <a id="intro"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>


Los algoritmos de aprendizaje supervisado generan un **modelo predictivo** a partir de los datos de entrada y de la salida esperada (la salida también se llama clase). La palabra **supervisado** viene de que un humano debe de haber creado o supervisado los valores de salida que el algoritmo tiene que aprender.

El conjunto de datos se denomina conjunto de entrenamiento y se utiliza para que el algoritmo de aprendizaje ajuste sus parámetros (por ejemplo los pesos y el umbral en el caso de las redes neuronales) y permita clasificar correctamente la salida a partir de los datos de entrada.

Existen dos tipos de algoritmos de aprendizaje supervisado:
- Algoritmos de clasificación: La salida o clase es un valor discreto. **El conjunto de datos "banana" que vamos a utilizar, es de clasificación.**
- Algoritmos de regresión: La salida o clase es un valor numérico. 

Estos conceptos y muchos más se ven en la parte teórica de la asignatura, así que los vamos a ver solo de pasada.

-------------------
En la siguiente celda podemos ver el esquema general de un algoritmo de aprendizaje supervisado y un ejemplo.

![esquema general](pics/esquemaGeneral-1.png)

# Vecinos más cercanos o k-NN <a id="vecinos"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

- Es el algoritmo de clasificación y regresión más simple.
- En la fase de entrenamiento simplemente almacena el conjunto de datos. Puede almacenarlo normalizado, luego veremos que es esto.
- Busca los $k$ vecinos más cercanos del ejemplo a predecir ($k$ es un parámetro, por ejemplo con $k=3$ significa que se usan los 3 ejemplos/filas más similares de entre los existentes en el conjunto de entrenamiento). 
- Pasos:
    - Se normalizan los atributos numericos. Restando el mínimo y dividiendo entre el rango.
    - Se calcula la distancia entre cada ejemplo, sumando las distancias de cada atributo. 
    - En atributos nominales (aquellos que no son números sino categorías), la distancia es 0 si son iguales o 1 si son distintos.
    - En numéricos es la diferencia de valores.
    - Se eligen los $k$ vecinos más cercanos. 
    - Se predice la moda (clasificación) o la media (regresión) de las clases de los ejemplos más cercanos.

Idea general del funcionamiento de vecinos más cercanos.

![knn](pics/knnk.png)

En la figura podemos ver las áreas de influencia o fronteras de decisión.

Todas las regiones de color morado indican que el algoritmo entrenado prediciría la clase "morado" para un ejemplo de test localizado en esa casilla.

Nota: Los ejemplos de test, son los que se usan para evaluar como de bien funciona un algoritmo de clasificación o regresión.

# Selección de instancias <a id="instance-selection"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

En la fase de preprocesado de los datos, una estrategia que puede repercutir positivamente en el rendimiento de los modelos de aprendizaje, es la **selección de instancias**.

La idea de la selección de instancias es mantener los ejemplos más útiles para el proceso de aprendizaje, y descartar aquellos que carezcan de utilidad e incluso puedan influir de forma negativa.

![enn](pics/enn.png)

## ENN (Vecinos más cercanos editados)

El algoritmo conocido como ENN hace un filtrado de instancias (ejemplos) teniendo en cuenta la idea de vecinos más cercanos.

- Dado un ejemplo, se buscan sus $k$ vecinos más cercanos.
- Si la clase del ejemplo no se corresponde con la clase mayoritaria de sus $k$ vecinos, ese ejemplo se suprime.

# Tareas de la practica <a id="tareas"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

En esta práctica se va a crear una clase llamada "EKNN" donde se deben implementar 4 métodos:
- vecinos(x,X,y).
- filtradoENN(X,y).
- entrenar(X,y). 
- predecir(X).

----------

- $x$ son los valores de atributos que definen a un ejemplo o individuo
- $X$ es un conjunto de ejemplos o individuos, cada uno de los cuales viene definido por los valores de sus atributos.
- $y$ son los valores que tratamos de predecir. Son la clase del invididuo.

In [2]:
import numpy as np
import pandas as pd
df=pd.read_csv('datasets/banana.csv')

# head saca los 5 primeros valores.
df.head()

,attr1,attr2,clase
0,0.431573,0.592162,-1
1,0.421647,0.330900,1
2,0.707009,0.443339,-1
3,0.648684,0.810578,-1
4,0.264226,0.211079,1


Cada ejemplo del conjunto de datos **"banana"** está formado por dos atributos (**att1**, y **att2**).

La clase a predecir puede tomar **dos** posibles valores (problema de clasificación binaria): **{-1, 1}**.

In [3]:
# y son los valores de la columna clase
# X son todas las columnas menos clase
y = df["clase"].values
X = df.drop("clase",axis=1).values

In [4]:
# 5 primeras filas
display(y[:5])
display(X[:5])

array([-1,  1, -1, -1,  1], dtype=int64)

array([[0.431573, 0.592162],
       [0.421647, 0.3309  ],
       [0.707009, 0.443339],
       [0.648684, 0.810578],
       [0.264226, 0.211079]])

Dividimos $X$ e $y$ en dos partes. Una parte se usará para entrenar el algoritmo y otra para probar que tal funciona.

A la parte que se usa para entrenar el algoritmo se le suele denominar *train* o conjunto de entrenamiento y a la parte que se usa para probar como de bien o mal funciona se le denomina *test*.

In [5]:
from sklearn.model_selection import train_test_split

'''
Se usa una función de Sklearn que también veremos más adelante en el temario

El par X_train, y_train son los atributos y clases del conjunto de entrenamiento (70% de los ejemplos)
El par X_test, y_test son los atributos y clases del conjunto de test (30% de los ejemplos)

'''
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)
print("X_train: ", np.shape(X_train))
print("X_test: ", np.shape(X_test))
print("y_train: ", np.shape(y_train))
print("y_test: ", np.shape(y_test))

X_train:  (927, 2)
X_test:  (398, 2)
y_train:  (927,)
y_test:  (398,)


## Esqueleto de la clase y el código que se pide


A continuación el esqueleto de lo que se pide. El alumno usar el código posterior como referencia y completarlo más abajo en una celda de tipo código.


```Python
class EKNN:

    def __init__(self, k):
        self.k = k
        
    def aprenderEstadisticas(self, X):
        self.minimo = X.min(axis=0)
        self.maximo = X.max(axis=0)
        
    def normalizar(self, X):
        try:
            return (X-self.minimo)/(self.maximo-self.minimo)
        except AttributeError:
            print("Aun no se ha calculado el mínimo y el máximo del conjunto de entrenamiento!")
            raise
        
    def vecinos(self, x, X, y):
        #completa este método
        pass
    
    def filtradoENN(self, X, y):
        #completa este método
        pass
    
    def entrenar(self,X,y):
        # completa este método
        pass
    
    def predecir(self,X):        
        valores_predichos = None
        X_norm = self.normalizar(X)
        # completa este método
        # valores_predichos debe de ser un array con las predicciones de cada uno de los ejemplos de X
        
        return valores_predichos
    
    
# Crea knn con 3 vecinos    
eknn = EKNN(3)
# entrena con el conjunto de entrenamiento
eknn.entrenar(X_train,y_train)    
# lo prueba con el conjunto de test
predicciones = eknn.predecir(X_test)

# calcula la precisión
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicciones)

```

Los métodos deberian tener control de errores. Por ejemplo si se usa *predecir* antes de haber invocado *entrenar* debería sacar un mensaje de error.

## Pistas

En el método de **vecinos** habría que:
- Obtener un array de distancias de $x$ a cada ejemplo de $X$.
    - La distancia se calcula sumando las diferencias en valor absoluto a cada uno de los atributos.
- Encontrar y retornar los $k$ ejemplos (índices) con distancias menores usando **argsort**.

En el método de **filtradoENN** habría que:
- Crear unos atributos en la clase $X$ e $y$ e inicializarlos con todos los ejemplos de entrenamiento.
- Recorrer todos los ejemplos de los nuevos atributos, y decidir si deben ser suprimidos o no.
- Hay que seguir una estrategia decremental, es decir, el array de ejemplos va disminuyendo a medida que filtramos.

En el método de **entrenar** habría que:
- Calcular el mínimo y el máximo del conjunto de entrenamiento (`aprenderEstadisticas`)
- Normalizar X_train a partir del mínimo y máximo calculados (`normalizar`)
- Filtrar X_train implementando `filtradoENN`.


En el método de **predecir**:
- Normalizar los ejemplos a predecir (X\_test). Usando los máximos y mínimos obtenidos en entrenar (`normalizar`).
- Obtener un array de distancias de cada ejemplo de X normalizado a cada ejemplo de X\_test.
    - La distancia se calcula sumando las diferencias en valor absoluto a cada uno de los atributos.
- Encontrar los $k$ ejemplos con distancias menores usando **argsort**
- Devolver la media de valores $y$ para esos ejemplos.

In [40]:
class EKNN:

    def __init__(self, k):
        self.k = k
        self.minimo = 0
        self.maximo = 0
        self.normalized = 0
        self.trained = False

    def aprenderEstadisticas(self, X):
        self.minimo = X.min(axis=0)
        self.maximo = X.max(axis=0)

    def normalizar(self, X):
        try:
            return (X-self.minimo)/(self.maximo-self.minimo)
        except AttributeError:
            print("Aun no se ha calculado el mínimo y el máximo del conjunto de entrenamiento!")
            raise

    
    def vecinos(self, x, X):
        distances = []
        neighbors = list()
        dist = 0
        print("x:", np.shape(x))
        print("X:", np.shape(X))
        for i in range(len(x)):
            dist += abs(X[i]-x[i])
            distances.append((X[i], dist))
            distSort = np.argsort(distances[i])[:self.k]
        for k in range(self.k):
            neighbors.append(distSort[k][0])
        return neighbors

    def filtradoENN(self, X, y):
        #completa este método
        pass

    def entrenar(self,X,y):
        self.aprenderEstadisticas(X)
        self.normalized = self.normalizar(X)
        self.y = y
        self.trained = True

    def predecir(self,X):        
        valores_predichos = None
        X_norm = self.normalizar(X)
        distances = []
        # completa este método
        # valores_predichos debe de ser un array con las predicciones de cada uno de los ejemplos de X
        if (not self.trained):
            print("train first!")
            return None
        if (self.k >= len(self.normalized)):
            print("k >= len(x_train) choose smaller k")
            return None
        
        #check arguments Datatype
        datatype = None
        if (isinstance(X_norm, pd.core.frame.DataFrame)):
            datatype = "panda"
        elif (isinstance(X_norm, np.ndarray)):
            datatype = "array"
        else:
            print("wrong datatype")
            return None
        
        #calculate distanc
        neighboors = self.vecinos(X_norm, self.normalized)
        print("neigh:", neighboors)
        valores_predichos = np.zeros(len(X))
        for i in range(len(X)):
            for j in range(self.k):
                valores_predichos[i] += self.y[neighboors[j]] 
            valores_predichos[i] = valores_predichos[i]/self.k
        return valores_predichos
    

# Crea knn con 3 vecinos
eknn = EKNN(3)
# entrena con el conjunto de entrenamiento
eknn.entrenar(X_train,y_train)    
# lo prueba con el conjunto de test
predicciones = eknn.predecir(X_test)

# calcula la precisión
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicciones)

x: (398, 2)
X: (927, 2)


IndexError: index 2 is out of bounds for axis 0 with size 2

In [39]:
class EKNN:

    def __init__(self, k):
        self.k = k
        self.minimum = 0
        self.maximum = 0
        self.normalised = 0
        self.trained = False

    def aprenderEstadisticas(self, X):
        self.minimo = X.min(axis=0)
        self.maximo = X.max(axis=0)

    def normalizar(self, X):
        try:
            return (X-self.minimum)/(self.maximum-self.minimum)
        except AttributeError:
            print("Aun no se ha calculado el mínimo y el máximo del conjunto de entrenamiento!")
            raise

    def vecinos(self, x, X, y):
        #completa este método
        distances = []
        length_X = self.normalizar(X)
        length_x = self.normalizar(x)
        for i in range(length_X):
            distances.append([])
            for j in range(length_x):
                distances[i].append(sum(abs(x[i]-X[j])))
        return np.argsort(distances)[:self.k]
            

    def filtradoENN(self, X, y):
        #completa este método
        self.y = y_train
        self.X = X_train

    def entrenar(self,X,y):
        # completa este método
        self.minimum = X.min(axis=0)
        self.maximum = X.max(axis=0)
        self.aprenderEstadisticas(X)
        self.normalised = self.normalizar(X)
        self.y = y
        self.trained = True

    def predecir(self,X):        
        if (not self.trained):
            print("train first!")
            return None
        if (self.k >= len(self.normalised)):
            print("k >= len(x_train) choose smaller k")
            return None

        normalisedX = self.normalizar(X)
        distance = []
        
        #check arguments Datatype
        datatype = None
        if (isinstance(normalisedX, pd.core.frame.DataFrame)):
            datatype = "panda"
        elif (isinstance(normalisedX, np.ndarray)):
            datatype = "array"
        else:
            print("wrong datatype")
            return None
        
        #calculate distance
        for i in range(len(normalisedX)):
            distance.append([])
            for j in range(len(self.normalised)):
                if(datatype == "panda"):
                    distance[i].append(sum(abs(normalisedX.iloc[i]-self.normalised.iloc[j])))
                else:
                    distance[i].append(sum(abs(normalisedX[i]-self.normalised[j])))
        
        #calculate predictions
        predictedValues = np.zeros(len(X))
        for i in range(len(X)):
            idx = np.argsort(distance[i])[:self.k]
            for j in range(self.k):
                predictedValues[i] += self.y[idx[j]] 
            predictedValues[i] = predictedValues[i]/self.k    
        return predictedValues


# Crea knn con 3 vecinos    
eknn = EKNN(3)
# entrena con el conjunto de entrenamiento
eknn.entrenar(X_train,y_train)    
# lo prueba con el conjunto de test
predicciones = eknn.predecir(X_test)
# calcula la precisión
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicciones.round())

0.7713567839195979

# Condiciones de entrega <a id="entrega"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

- La entrega debe realizarse a través de UBUVirtual antes del **20 de marzo de 2020 a las 23:59**.
- Se debe entregar un .zip con este proyecto jupyter notebook con la solución.
- Adicionalmente (no es obligatorio, pero se valorará positivamente) se puede hacer un informe comparando la solución obtenida con vuestra implementación, contra la obtenida con un workflow en KNIME donde utilicéis el nodo K Nearest Neighbor, razonando si el uso de la estrategia de selección de instancias afecta positivamente al rendimiento del KNN.